# HEDDEx Inference

This notebook is meant to provide an interactive API for inference on the joint model to predict terms, abbreviations, symbols and their definitions etc. There are two meta tasks that can be accomplished by these models.
### Meta Task 1 (Slot Prediction)
- Intent prediction per sentence and Slot prediction per token of the sentence for the tasks the model was trained on (AI2020, DocDef, W00, etc.)
- Example input : `The input to the matrix [[M]] is a vector [[v]] of specified lengths.`
- Example output : {''}

### Meta Task 2 (Query based Prediction)
- These models are trained to predict the nicknames for symbols (DocDef dataset), given a special query token around the symbol whose nickname we want predicted
- Example input : `The input to the matrix [[M]] is a vector </s> [[v]] </s> of specified lengths.`
- Example output : {''}    

In [1]:
!which python

/home/nuwandavek/anaconda3/envs/scholarphi/bin/python


In [2]:
%load_ext autoreload
%autoreload 2
import sys, os
sys.path.append('../')
from model import JointRoberta
from transformers import HfArgumentParser, AutoTokenizer
from configuration import (DataTrainingArguments,ModelArguments,TrainingArguments,)
from utils import get_joint_labels
from trainer_joint import JointTrainer

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [8]:
# Make sure the variables here are correct
# models_dir_path = './models/DocDef2+AI2020+W00_model'
# models_dir_path = "./models/joint_abbrexp_termdef_symnick"
models_dir_path = "./models/AI2020_model"
# task = "DocDef2+AI2020+W00"
task = "AI2020"

In [9]:
parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainingArguments))
model_args, data_args, training_args = parser.parse_args_into_dataclasses([
    "--model_name_or_path",
    "roberta-large",
    "--data_dir",
    models_dir_path,
    "--task",
    task,
    "--output_dir",
    f"{models_dir_path}/roberta-large",
    "--use_crf",
    "--use_heuristic",
    "--use_pos",
    "--use_np",
    "--use_vp",
    "--use_entity",
    "--use_acronym",
    "--do_train",
    "--per_device_train_batch_size",
    "4",
    "--per_device_eval_batch_size",
    "4",
    "--max_seq_len",
    "80"])
training_args.output_dir = "{}{}{}{}{}{}".format(
            training_args.output_dir,
            "_pos={}".format(training_args.use_pos)
            if training_args.use_pos
            else "",
            "_np={}".format(training_args.use_np) if training_args.use_np else "",
            "_vp={}".format(training_args.use_vp) if training_args.use_vp else "",
            "_entity={}".format(training_args.use_entity)
            if training_args.use_entity
            else "",
            "_acronym={}".format(training_args.use_acronym)
            if training_args.use_acronym
            else "",
)
os.environ["WANDB_DISABLED"] = "1"

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_args.model_name_or_path)
intent_label_dict = get_joint_labels(training_args, "intent_label")
slot_label_dict = get_joint_labels(training_args, "slot_label")
pos_label_lst = get_joint_labels(training_args, "pos_label")

model = JointRoberta.from_pretrained(
    training_args.output_dir,
    args=training_args,
    intent_label_dict=intent_label_dict,
    slot_label_dict=slot_label_dict,
    pos_label_lst=pos_label_lst,
    tasks=task.split('+'),
)

trainer = JointTrainer([training_args, model_args, data_args,],model,tokenizer=tokenizer)



loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-large-config.json from cache at /home/nuwandavek/.cache/torch/transformers/c22e0b5bbb7c0cb93a87a2ae01263ae715b4c18d692b1740ce72cacaa99ad184.2d28da311092e99a05f9ee17520204614d60b0bfdb32f8a75644df7737b6a748
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "type_vocab_size": 1,
  "vocab_size": 50265
}

loading file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-large-vocab.json from cache at /home/nuwandavek/.cache/torch/transformers/1ae1f5b6e2b22b25

In [11]:
text = "In order to dynamically learn filters and features we look to Convolutional Neural Networks (CNNs) which have shown their dominance in computer vision Citation ( CITATION ) ."
print(trainer.predict_batch([text]))

Converting example 0 of 1
*** Example ***
guid: one
tokens: <s> In order to d ynam ically learn fil ters and features we look to Con v olution al Ne ural Net works ( CNN s ) which have shown their dom inance in computer vision C itation ( C IT ATION ) . </s>
input_ids: 0 1121 10337 560 417 39116 3435 38229 18419 2696 463 46076 1694 13724 560 9157 705 23794 337 14563 9799 15721 11655 1640 16256 29 43 5488 11990 44270 25017 12623 13598 179 36327 14675 347 12257 1640 347 2068 6034 43 4 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
intent_label: 1 (id = 1)
slot_labels: 0 1 1 1 1 0 0 1 1 0 1 1 1 1 1 1 0 0 0 1 0 1 0 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
PyTorch: setting up devices


({'AI2020': [1]}, {'AI2020': [['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'DEF', 'DEF', 'DEF', 'O', 'TERM', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'DEF', 'DEF', 'DEF', 'O', 'TERM', 'O', 'O']]}, {'AI2020': [[0.99997866, 0.999969, 0.99998283, 0.99994326, 0.9999665, 0.9999585, 0.99998, 0.99997663, 0.99997556, 0.9999653, 0.99997437, 0.9998549, 0.99977964, 0.9998603, 0.99975663, 0.99949634, 0.9999597, 0.9999745, 0.9999821, 0.99997866, 0.9999826, 0.9999548, 0.9999634, 0.9771666, 0.9833735, 0.9893351, 0.9998086, 0.9968988, 0.99988735, 0.95342153]]})


In [12]:
text = "In order to dynamically learn filters and features we look to Convolutional Neural Networks (CNNs) which have shown their dominance in computer vision"
print(trainer.predict_batch([text]))

Converting example 0 of 1
*** Example ***
guid: one
tokens: <s> In order to d ynam ically learn fil ters and features we look to Con v olution al Ne ural Net works ( CNN s ) which have shown their dom inance in computer vision </s>
input_ids: 0 1121 10337 560 417 39116 3435 38229 18419 2696 463 46076 1694 13724 560 9157 705 23794 337 14563 9799 15721 11655 1640 16256 29 43 5488 11990 44270 25017 12623 13598 179 36327 14675 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
intent_label: 1 (id = 1)
slot_labels: 0 1 1 1 1 0 0 1 1 0 1 1 1 1 1 1 0 0 0 1 0 1 0 1 1 0 1 1 1 1 1 1 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


({'AI2020': [1]}, {'AI2020': [['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'DEF', 'DEF', 'DEF', 'O', 'TERM', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]}, {'AI2020': [[0.9999759, 0.9999647, 0.9999778, 0.9999305, 0.999961, 0.99995065, 0.9999795, 0.9999751, 0.9999726, 0.9999577, 0.9999759, 0.9998479, 0.9998282, 0.99986744, 0.9997521, 0.99921274, 0.9999627, 0.99997413, 0.99998164, 0.9999789, 0.9999814, 0.99994683, 0.9999844, 0.9999691, 0.9999796]]})
